In [1]:
import yfinance as yf
import numpy as np 
import pandas as pd
import requests
import json
import copy

In [2]:
url = "https://api.usfundamentals.com/v1/companies/xbrl"

In [3]:
contracts = pd.read_csv("data/contracts_data.csv")
contracts.head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
0,24315059,HT940216C0001,HUMANA GOVERNMENT BUSINESS INC,47952334646.21,NaN,IGF::OT::IGF,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Defense,Defense Health Agency,2016-08-01,2025-12-31,1068e466-707d-6b6f-367f-0255fab79b8c-C,NaN,1173,department-of-defense,CONT_AWD_HT940216C0001_9700_-NONE-_-NONE-
1,12178950,DEAC3243AL00036,"REGENTS OF THE UNIVERSITY OF CALIFORNIA, THE",35271576063.18,28908000.0,M&O OF LANL BR OF U OF CA,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Energy,Department of Energy,1978-09-30,2006-05-31,dfae65bd-32fe-bce0-389c-2867fa9fa04e-C,NaN,930,department-of-energy,CONT_AWD_DEAC3243AL00036_8900_-NONE-_-NONE-
2,25882618,N0001917C0001,LOCKHEED MARTIN CORPORATION,35135840073.2,933036846.29,LRIP LOT 12 ADVANCE ACQUISITION CONTRACT,DEFINITIVE CONTRACT,"['N', 'Q']",11234642.22,1741868.78,...,NaN,Department of Defense,Defense Contract Management Agency,2017-11-17,2031-03-31,6cf5fb1b-4988-d087-5dc1-70939d8fc6c4-C,NaN,1173,department-of-defense,CONT_AWD_N0001917C0001_9700_-NONE-_-NONE-
3,12179201,DEAC5207NA27344,"LAWRENCE LIVERMORE NATIONAL SECURITY, LLC",34868265435.3,11025314932.05,TAS::89 0240::TAS THIS PERFORMANCE-BASED MANAG...,DEFINITIVE CONTRACT,"['AAA', 'AAB', 'N', 'Q', 'Z']",12343732.4,12341690.08,...,17359702.05,Department of Energy,Department of Energy,2007-05-09,2026-09-30,6d990129-9566-ff86-e9ae-7c6eeef1df7c-C,NaN,930,department-of-energy,CONT_AWD_DEAC5207NA27344_8900_-NONE-_-NONE-
4,25881000,N0001902C3002,LOCKHEED MARTIN CORPORATION,34132621786.26,29313885.86,200204!008532!1700!AF600 !NAVAL AIR SYSTEMS CO...,DEFINITIVE CONTRACT,"['N', 'Q']",-624555.7,1530623.99,...,NaN,Department of Defense,Department of the Navy,2001-10-26,2022-12-16,6cf5fb1b-4988-d087-5dc1-70939d8fc6c4-C,NaN,1173,department-of-defense,CONT_AWD_N0001902C3002_9700_-NONE-_-NONE-


In [17]:
contracts["Start Date"]

0        2016-08-01
1        1978-09-30
2        2017-11-17
3        2007-05-09
4        2001-10-26
            ...    
35543    2020-03-31
35544    2019-03-14
35545    2019-01-14
35546    2013-01-06
35547    2011-06-13
Name: Start Date, Length: 35548, dtype: object

In [31]:
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test.sort_values(by="Start Date")
contracts_test

/tmp/ipykernel_45302/4130340711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])


,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
18290,12178473,DEAC1276SN10559,UNC INC,46700508.0,NaN,FABRICATE NAVAL REACTOR CORES,DCA,NaN,NaN,NaN,...,NaN,Department of Energy,Department of Energy,1973-08-24,2012-09-30,69f389f4-61e3-931d-5f27-148e12234c4f-C,NaN,930,department-of-energy,CONT_AWD_DEAC1276SN10559_8900_-NONE-_-NONE-
6,12178133,DEAC0576RL01830,BATTELLE MEMORIAL INSTITUTE,27983582293.19,5344341098.6,BATTELLE MEMORIAL INSTITUTE/PACIFIC NORTHWEST ...,DEFINITIVE CONTRACT,"['3', 'AAA', 'AAB', 'C', 'N', 'Q', 'R', 'Z']",6259201.57,6259201.57,...,34336489.51,Department of Energy,Department of Energy,1978-09-15,2027-09-30,74457bb7-b18d-9762-896e-e575e7a4ed82-C,NaN,930,department-of-energy,CONT_AWD_DEAC0576RL01830_8900_-NONE-_-NONE-
1,12178950,DEAC3243AL00036,"REGENTS OF THE UNIVERSITY OF CALIFORNIA, THE",35271576063.18,28908000.0,M&O OF LANL BR OF U OF CA,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Energy,Department of Energy,1978-09-30,2006-05-31,dfae65bd-32fe-bce0-389c-2867fa9fa04e-C,NaN,930,department-of-energy,CONT_AWD_DEAC3243AL00036_8900_-NONE-_-NONE-
27,12177779,DEAC0276SF00515,THE LELAND STANFORD JUNIOR UNIVERSITY,13703875273.19,2569980276.56,OPERATION AND MAINTENANCE ...,DEFINITIVE CONTRACT,"['N', 'Q', 'R', 'Z']",3639810.2,3639810.2,...,7950.6,Department of Energy,Department of Energy,1978-11-20,2027-09-30,0c13d643-4334-38ed-1952-0db7e2ac5e56-C,NaN,930,department-of-energy,CONT_AWD_DEAC0276SF00515_8900_-NONE-_-NONE-
1044,12178667,DEAC2481NE44139,WEST VALLEY NUCLEAR SERVICES COMPANY LLC,700513818.01,NaN,PRIME OPERATING CONTRACTOR FORTHE WEST VALLEY ...,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Energy,Department of Energy,1981-08-24,2007-07-30,0746598b-a46e-92e8-d403-f1f1f3792691-C,NaN,930,department-of-energy,CONT_AWD_DEAC2481NE44139_8900_-NONE-_-NONE-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12157,159768202,FA862523F3031,LOCKHEED MARTIN CORP,68642805.0,NaN,C-130J / DMS B1 GROUPING UCA,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2027-07-31,2027-07-31,4051d253-67f0-30bd-2c7f-5eb539533924-C,NaN,1173,department-of-defense,CONT_AWD_FA862523F3031_9700_FA862520D3000_9700
16760,155276901,FA863422F0044,THE BOEING COMPANY,50226731.76,NaN,F-15 -- RAPID RESPONSE DELIVERY ORDER,DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2027-07-31,2027-09-21,9ab85632-42cc-5fa4-2a9c-d734034ddd33-C,NaN,1173,department-of-defense,CONT_AWD_FA863422F0044_9700_FA863417D2696_9700
702,154090097,FA862522F3027,LOCKHEED MARTIN CORP,973687330.0,NaN,C-130J ADD AIRCRAFTS FY22 CONGRESSIONAL,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2027-09-30,2027-09-30,4051d253-67f0-30bd-2c7f-5eb539533924-C,NaN,1173,department-of-defense,CONT_AWD_FA862522F3027_9700_FA862520D3000_9700
1164,147366297,FA863422C2705,THE BOEING COMPANY,642930773.59,NaN,F-15/NO ACAT/JAPAN/UNDEFINITIZED CONTRACT ACTION,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2028-12-29,2028-11-30,9ab85632-42cc-5fa4-2a9c-d734034ddd33-C,NaN,1173,department-of-defense,CONT_AWD_FA863422C2705_9700_-NONE-_-NONE-


In [4]:
from groq import Groq

client = Groq(api_key="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")

In [5]:
def isPublic(company): 
    completion = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[
            {
                "role": "system",
                "content": "Identify publicly traded companies that have a positive correlation with a given company. consider other companies that would benefit from this company thriving, while excluding direct competitors who might not experience the same positive effect. Also, provide any parent or subsidiary companies that are publicly traded, including the company itself if applicable. \n\n# Required Information:\n- Given company name.\n\n# Details to Include:\n- List companies that have a positive correlation with the given company if it receives a large (>100M USD) government contract.\n- Include companies positively influenced by this event or that have a linked dependency, excluding direct competitors as they might not benefit under the same conditions.\n- Include parent or subsidiary companies of the given company, if they are publicly traded.\n- Include the given company itself if it is publicly traded.\n\n# Output Format:\n\nProvide a list ordered by correlation to the given company separated by a comma of their TICKERS ONLY. do not include further information or notes. Return N/A if the company is not publicly traded.\n\n[TICKER]\n\n"
            }, 
            {
                "role": "user",
                "content": f"is {company}"
            },
        ],
        temperature=1.12,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    full_response = ""
    for chunk in completion:
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            full_response+=(chunk.choices[0].delta.content)
    return full_response

In [38]:
filtered_df = contracts_test[
    (contracts_test["Start Date"] >= pd.Timestamp("2010-01-01")) & 
    (contracts_test["Start Date"] <= pd.Timestamp("2023-12-31"))
]
filtered_df

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
27549,4852070,1003,GENERAL MILLS INC,31977440.3,NaN,RESALE - FLOUR AND OTHER MISCELLANEOUS BAKING ...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Commissary Agency,2010-01-01,2010-03-31,29ba69f0-9ae9-608e-3dc9-b49cfe24774b-C,NaN,1173,department-of-defense,CONT_AWD_1003_9700_HDEC0109G3837_9700
25848,809813,0008,ROLLS-ROYCE CORPORATION,33949832.47,NaN,DELIVERY 0RDER 0008 FOR CONTRACTOR LOGISTICS S...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2010-01-01,2012-09-30,ae61fda0-8744-9361-a177-0ffcabb70c2e-C,NaN,1173.0,department-of-defense,CONT_AWD_0008_9700_W58RGZ06D0072_9700
7963,1162439,0016,DYNCORP INTERNATIONAL LLC,102704184.07,NaN,TIME AND MATERIAL LABOR,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2010-01-01,2011-06-30,45d836a2-4bbe-5707-782d-26be5f02fd93-C,NaN,1173,department-of-defense,CONT_AWD_0016_9700_FA810809D0004_9700
31680,4850615,1003,ARMY & AIR FORCE EXCHANGE SERVICE,28024087.63,NaN,RESALE - CIGARETTES,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Commissary Agency,2010-01-01,2010-03-31,6f2964c6-4882-9437-2b7f-73856fcff7f0-C,NaN,1173,department-of-defense,CONT_AWD_1003_9700_DECA0197GM0A2_9700
29208,63373,0001,"CACI, INC. - FEDERAL",30268704.22,NaN,BASE PERIOD,DO,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Navy,2010-01-01,2010-12-31,28eee911-76af-7147-f760-4ec5f90bec2d-C,NaN,1173,department-of-defense,CONT_AWD_0001_9700_N0018910DZ006_9700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33471,167179909,19AQMM24F0158,ATT MOBILITY LLC,26592999.0,NaN,TASK ORDER FOR WIRELESS SERVICES,DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of State,Department of State,2023-12-31,2024-12-30,c0aa4884-6621-8b31-0942-50047a976c83-C,NaN,315,department-of-state,CONT_AWD_19AQMM24F0158_1900_47QTCA19D00MV_4732
12241,155657225,FA862023F4002,L3HARRIS TECHNOLOGIES INTEGRATED SYSTEMS L.P.,68199108.13,NaN,BIG SAFARI,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2023-12-31,2023-12-31,14a893b4-32fd-654e-e810-f0ed061e33a2-C,NaN,1173,department-of-defense,CONT_AWD_FA862023F4002_9700_FA862019G4006_9700
13388,170062129,FA807524F0005,"KBR WYLE SERVICES, LLC",62364317.83,NaN,TECHNICAL ANALYSIS FOR THE DEPARTMENT OF THE A...,DELIVERY ORDER,['Q'],NaN,NaN,...,NaN,Department of Defense,Department of the Air Force,2023-12-31,2028-12-30,c1555398-ca16-7024-230c-c902d4f1a351-C,NaN,1173,department-of-defense,CONT_AWD_FA807524F0005_9700_FA807518D0015_9700
9595,166787550,12363N24C4002,"SIERRA TAHOE ENVIRONMENTAL MANAGEMENT, LLC",85643865.96,38615.47,SPIRIT G-Z STEWARDSHIP IRSC - 12363N24C4002 $8...,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Agriculture,Forest Service,2023-12-31,2031-12-31,c9446657-d5f1-02ea-a967-bf137c3b56bf-C,NaN,95,department-of-agriculture,CONT_AWD_12363N24C4002_12C2_-NONE-_-NONE-
